In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pickle
import os
import glob
from tqdm import tqdm

2023-07-04 14:50:27.151469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-04 14:50:27.151498: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = np.load("1_data/train_data.npy") 
label = np.load("1_data/train_label.npy")
print("data.shape=", data.shape)
print("label.shape=", label.shape)

data.shape= (3000, 1, 28, 28)
label.shape= (3000, 15)


In [3]:
# 文字種毎にデータ抽出、データ拡張
for i in range(15):
    index = np.where(label[:, i] == 1)

    data_c = data[index[0]]
    data_c = data_c.transpose(0, 2, 3, 1)

    # ImageDataGeneratorのオブジェクト生成
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.9,
#        zoom_range=0.1,
        height_shift_range=0.15,
        width_shift_range=0.15,
        rotation_range=20
    )
    # 生成後枚数
    num_image = 1000

    # 保存先ディレクトリの作成    
    save_dir = "1_data/out_%s/" % i
    os.mkdir(save_dir)

    # データ拡張
    g = datagen.flow(data_c, batch_size=1, save_to_dir=save_dir, save_format='png', save_prefix='out_a_from_npy_')
    for k in range(num_image):
        batches = g.next()

IMG_SIZE = 28
# True=Grayscale, False=RGB
COLOR = True
# Name to save
SAVE_FILE_NAME = 'SaveImages'
# shape File Name
if COLOR:
    SAVE_FILE_NAME = SAVE_FILE_NAME + '_' + str(IMG_SIZE) + 'Gray'
else:
    SAVE_FILE_NAME = SAVE_FILE_NAME + '_' + str(IMG_SIZE) + 'RGB'

merge_array = np.empty([0, 28, 28, 1])


for l in range(15):
    # Name to load images Folder
    DIR_NAME = '1_data/out_%s' % l

    # load images and reshape
    img_list = glob.glob(DIR_NAME + '/*.png')
    temp_img_array_list = []
    for img in tqdm(img_list):
        temp_img = keras.preprocessing.image.load_img(img, color_mode='grayscale' if COLOR else 'rgb',
                                                      target_size=(IMG_SIZE, IMG_SIZE))
        temp_img_array = keras.preprocessing.image.img_to_array(temp_img)
        temp_img_array_list.append(temp_img_array)

    temp_img_array_list = np.array(temp_img_array_list)

    print(temp_img_array_list.shape)

    merge_array = np.concatenate([merge_array, temp_img_array_list])

print(merge_array.shape)
merge_array = merge_array.transpose(0, 3, 1, 2)
np.save("new_data.npy", merge_array)

label_count = []

for i in range(15):
    path = "1_data/out_%s" % i
    files = os.listdir(path)
    count = len(files)
    label_count.append(count)


# 各ラベル作成
char_01 = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_02 = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_03 = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_04 = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_05 = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_06 = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
char_07 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
char_08 = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
char_09 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
char_10 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
char_11 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
char_12 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
char_13 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
char_14 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
char_15 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

for i in range(15):
    if i == 0:
        array_01 = np.tile(char_01, (label_count[i], 1))
    if i == 1:
        array_02 = np.tile(char_02, (label_count[i], 1))
    if i == 2:
        array_03 = np.tile(char_03, (label_count[i], 1))
    if i == 3:
        array_04 = np.tile(char_04, (label_count[i], 1))
    if i == 4:
        array_05 = np.tile(char_05, (label_count[i], 1))
    if i == 5:
        array_06 = np.tile(char_06, (label_count[i], 1))
    if i == 6:
        array_07 = np.tile(char_07, (label_count[i], 1))
    if i == 7:
        array_08 = np.tile(char_08, (label_count[i], 1))
    if i == 8:
        array_09 = np.tile(char_09, (label_count[i], 1))
    if i == 9:
        array_10 = np.tile(char_10, (label_count[i], 1))
    if i == 10:
        array_11 = np.tile(char_11, (label_count[i], 1))
    if i == 11:
        array_12 = np.tile(char_12, (label_count[i], 1))
    if i == 12:
        array_13 = np.tile(char_13, (label_count[i], 1))
    if i == 13:
        array_14 = np.tile(char_14, (label_count[i], 1))
    if i == 14:
        array_15 = np.tile(char_15, (label_count[i], 1))

label_data_mr = np.concatenate([array_01, array_02, array_03, array_04, array_05, array_06, array_07, array_08, array_09, array_10, array_11, array_12, array_13, array_14, array_15])

np.save("new_label.npy", label_data_mr)

100%|██████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5365.71it/s]


(1000, 28, 28, 1)


100%|██████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4976.31it/s]


(1000, 28, 28, 1)


100%|██████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4745.56it/s]


...

(1000, 28, 28, 1)


100%|██████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5124.19it/s]


(1000, 28, 28, 1)


100%|██████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5379.97it/s]


(1000, 28, 28, 1)
(15000, 28, 28, 1)


In [4]:
train_data = np.concatenate((np.load("1_data/train_data.npy"), np.load('new_data.npy')), axis=0)
train_label = np.concatenate((np.load("1_data/train_label.npy"), np.load('new_label.npy')), axis=0)

# 正規化
train_data = train_data / train_data.max()
train_data = train_data.astype('float32')

print("train_data.shape=",train_data.shape)

train_data.shape= (18000, 1, 28, 28)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label,
                                                    test_size=0.3, random_state=1234,
                                                    shuffle=True
                                                    )
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12600, 1, 28, 28) (5400, 1, 28, 28) (12600, 15) (5400, 15)


In [6]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12600, 28, 28, 1) (5400, 28, 28, 1) (12600, 15) (5400, 15)


In [7]:
tf.config.set_visible_devices([], 'GPU')

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.Dense(units=64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.Dense(units=15, activation='softmax')
])


# モデルのコンパイル
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])



# モデルの概要の表示
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0

2023-07-04 14:51:02.538399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-04 14:51:02.538464: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-04 14:51:02.538501: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (masaki-Precision-Tower-5810): /proc/driver/nvidia/version does not exist
2023-07-04 14:51:02.563786: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
history = model.fit(X_train, y_train, epochs=50, batch_size=50, validation_data=(X_test, y_test))

Epoch 1/50
252/252 [==============================] - 4s 14ms/step - loss: 0.7918 - accuracy: 0.7933 - val_loss: 4.3901 - val_accuracy: 0.0670
Epoch 2/50
252/252 [==============================] - 3s 14ms/step - loss: 0.1989 - accuracy: 0.9492 - val_loss: 1.3419 - val_accuracy: 0.5796
Epoch 3/50
252/252 [==============================] - 4s 14ms/step - loss: 0.0798 - accuracy: 0.9825 - val_loss: 0.6458 - val_accuracy: 0.7946
Epoch 4/50
252/252 [==============================] - 3s 14ms/step - loss: 0.0427 - accuracy: 0.9914 - val_loss: 0.3822 - val_accuracy: 0.8743
Epoch 5/50
252/252 [==============================] - 3s 14ms/step - loss: 0.0264 - accuracy: 0.9952 - val_loss: 0.3414 - val_accuracy: 0.8970
Epoch 6/50
252/252 [==============================] - 4s 14ms/step - loss: 0.0146 - accuracy: 0.9979 - val_loss: 0.3366 - val_accuracy: 0.9020
Epoch 7/50
252/252 [==============================] - 3s 14ms/step - loss: 0.0103 - accuracy: 0.9989 - val_loss: 0.5062 - val_accuracy: 0.8498

In [9]:
# モデルの保存
model.save('trained_model.h5')